In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Paths
train_path = r"C:\Users\yamin\python programs\My_Project\Dataset\Train"
valid_path = r"C:\Users\yamin\python programs\My_Project\Dataset\Test"
model_save_path = "alexnet_currency_model.pth"

# Parameters
batch_size = 32
num_epochs = 10
learning_rate = 0.0001

# Data Preprocessing and Augmentation
train_transforms = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(train_path, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_path, transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Load Pretrained AlexNet Model
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Modify the classifier for the custom dataset
num_classes = len(train_dataset.classes)
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=learning_rate)

# Training Loop
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%")

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss/len(valid_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%\n")

# Train and Save Model
train_model(alexnet, train_loader, valid_loader, criterion, optimizer, num_epochs)
torch.save(alexnet.state_dict(), model_save_path)
print("Model training completed and saved.")


Epoch 1/10, Loss: 1.8224, Train Accuracy: 34.64%
Validation Loss: 1.0783, Validation Accuracy: 72.47%

Epoch 2/10, Loss: 0.7762, Train Accuracy: 81.70%
Validation Loss: 0.3924, Validation Accuracy: 92.70%

Epoch 3/10, Loss: 0.2617, Train Accuracy: 93.46%
Validation Loss: 0.1548, Validation Accuracy: 94.94%

Epoch 4/10, Loss: 0.0818, Train Accuracy: 98.04%
Validation Loss: 0.1177, Validation Accuracy: 94.94%

Epoch 5/10, Loss: 0.0625, Train Accuracy: 98.04%
Validation Loss: 0.0752, Validation Accuracy: 96.63%

Epoch 6/10, Loss: 0.0079, Train Accuracy: 100.00%
Validation Loss: 0.2962, Validation Accuracy: 95.51%

Epoch 7/10, Loss: 0.0889, Train Accuracy: 97.39%
Validation Loss: 0.0927, Validation Accuracy: 97.19%

Epoch 8/10, Loss: 0.0981, Train Accuracy: 96.08%
Validation Loss: 0.0489, Validation Accuracy: 98.31%

Epoch 9/10, Loss: 0.0198, Train Accuracy: 98.69%
Validation Loss: 0.0911, Validation Accuracy: 97.75%

Epoch 10/10, Loss: 0.0813, Train Accuracy: 98.04%
Validation Loss: 0.069

In [2]:
!pip3 install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from tkinter import Tk, Label, Button, filedialog
from PIL import Image, ImageTk  # Use ImageTk for Tkinter compatibility

# Define Paths for Model
model_path = "alexnet_currency_model.pth"  # Path to your saved model

# Load Pretrained AlexNet Model with specified weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model architecture with the pretrained weights
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Modify the final classifier layer for custom classes (Real or Fake)
num_classes = 2  # "Real" or "Fake"
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Move model to the correct device (GPU or CPU)
alexnet = alexnet.to(device)

# Try to load the model state dict from the saved file
try:
    alexnet.load_state_dict(torch.load(model_path, map_location=device))
    alexnet.eval()
    print("Model loaded successfully!")
except RuntimeError as e:
    print(f"Error loading model: {e}")
    print("Ensure the model architecture is the same as the saved model.")

# Image Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize to AlexNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# GUI Application
class CurrencyNoteClassifier:
    def __init__(self, root):
        self.root = root
        self.root.title("Currency Note Classifier")
        self.root.geometry("500x400")
        
        self.label = Label(root, text="Upload a Currency Note Image", font=("Arial", 14))
        self.label.pack(pady=20)
        
        self.upload_button = Button(root, text="Upload Image", command=self.upload_image, font=("Arial", 12))
        self.upload_button.pack(pady=10)
        
        self.result_label = Label(root, text="", font=("Arial", 16), fg="blue")
        self.result_label.pack(pady=20)
        
        self.image_label = Label(root)
        self.image_label.pack()

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            # Use PIL to open the image and convert it to a Tkinter-compatible format
            img = Image.open(file_path)
            img.thumbnail((250, 250))  # Resize for display

            # Convert the image to a format Tkinter understands (PhotoImage)
            img_tk = ImageTk.PhotoImage(img)
            
            # Display the image in the GUI
            self.image_label.image = img_tk
            self.image_label.configure(image=self.image_label.image)
            
            # Preprocess the image and predict
            result = self.predict_currency(img)
            self.result_label.configure(text=f"Prediction: {result}")

    def predict_currency(self, image):
        # Preprocess the image
        image = preprocess(image).unsqueeze(0).to(device)
        
        # Predict using the model
        with torch.no_grad():
            outputs = alexnet(image)
            _, predicted = torch.max(outputs, 1)
        
        # Define class labels
        classes = ["Real", "Fake"]
        return classes[predicted.item()]

# Initialize and run GUI
root = Tk()
app = CurrencyNoteClassifier(root)
root.mainloop()


C:\Users\yamin\AppData\Local\Temp\ipykernel_6932\2429377666.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  alexnet.load_state_dict(torch.load(model_path, map_location=

Error loading model: Error(s) in loading state_dict for AlexNet:
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([7, 4096]) from checkpoint, the shape in current model is torch.Size([2, 4096]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([2]).
Ensure the model architecture is the same as the saved model.


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from tkinter import Tk, Label, Button, filedialog
from PIL import Image, ImageTk  # Use ImageTk for Tkinter compatibility

# Define Paths for Model
model_path = "alexnet_currency_model.pth"  # Path to your saved model

# Load Pretrained AlexNet Model with specified weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model architecture with the pretrained weights
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Modify the final classifier layer for custom classes (Real or Fake)
num_classes = 2  # "Real" or "Fake"
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, num_classes)

# Move model to the correct device (GPU or CPU)
alexnet = alexnet.to(device)

# Try to load the model state dict from the saved file
try:
    alexnet.load_state_dict(torch.load(model_path, map_location=device))
    alexnet.eval()
    print("Model loaded successfully!")
except RuntimeError as e:
    print(f"Error loading model: {e}")
    print("Ensure the model architecture is the same as the saved model.")

# Image Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize to AlexNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# GUI Application
class CurrencyNoteClassifier:
    def __init__(self, root):
        self.root = root
        self.root.title("Currency Note Classifier")
        self.root.geometry("600x500")  # Increased window size to fit more content
        
        self.label = Label(root, text="Upload a Currency Note Image", font=("Arial", 14))
        self.label.pack(pady=20)
        
        self.upload_button = Button(root, text="Upload Image", command=self.upload_image, font=("Arial", 12))
        self.upload_button.pack(pady=10)
        
        self.result_label = Label(root, text="", font=("Arial", 16), fg="blue")
        self.result_label.pack(pady=20)
        
        self.image_label = Label(root)
        self.image_label.pack(pady=10)

        self.grayscale_label = Label(root)
        self.grayscale_label.pack(pady=10)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            # Use PIL to open the image
            img = Image.open(file_path)
            img.thumbnail((250, 250))  # Resize for display

            # Convert the image to a format Tkinter understands (PhotoImage)
            img_tk = ImageTk.PhotoImage(img)
            
            # Display the original image in the GUI
            self.image_label.image = img_tk
            self.image_label.configure(image=self.image_label.image)

            # Convert the image to grayscale
            grayscale_img = img.convert('L')
            grayscale_img.thumbnail((250, 250))  # Resize grayscale image for display

            # Convert grayscale image to a format Tkinter understands
            grayscale_img_tk = ImageTk.PhotoImage(grayscale_img)
            
            # Display the grayscale image in the GUI
            self.grayscale_label.image = grayscale_img_tk
            self.grayscale_label.configure(image=self.grayscale_label.image)

            # Preprocess the original image and predict
            result = self.predict_currency(img)
            self.result_label.configure(text=f"Prediction: {result}")

    def predict_currency(self, image):
        # Preprocess the image for model input
        image = preprocess(image).unsqueeze(0).to(device)
        
        # Predict using the model
        with torch.no_grad():
            outputs = alexnet(image)
            _, predicted = torch.max(outputs, 1)
        
        # Define class labels
        classes = ["Real", "Fake"]
        return classes[predicted.item()]

# Initialize and run GUI
root = Tk()
app = CurrencyNoteClassifier(root)
root.mainloop()


C:\Users\yamin\AppData\Local\Temp\ipykernel_6932\3342861601.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  alexnet.load_state_dict(torch.load(model_path, map_location=

Error loading model: Error(s) in loading state_dict for AlexNet:
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([7, 4096]) from checkpoint, the shape in current model is torch.Size([2, 4096]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([2]).
Ensure the model architecture is the same as the saved model.


In [5]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install torch